*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


# Load libraries

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 584.5 MB 3.8 kB/s 
     |████████████████████████████████| 5.9 MB 41.6 MB/s 
     |████████████████████████████████| 439 kB 54.2 MB/s 
     |████████████████████████████████| 1.7 MB 46.2 MB/s 
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.11.0-dev20220903


# Load dataset

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-09-03 13:26:31--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2022-09-03 13:26:32 (8.45 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-09-03 13:26:32--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

# Explore and label dataset

In [37]:
train_data = pd.read_csv(train_file_path, sep='\t', names = ["Label", "Message"])
train_data.head()

,Label,Message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [38]:
train_data.tail()

,Label,Message
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...
4178,ham,tell my bad character which u dnt lik in me. ...


In [39]:
test_data = pd.read_csv(test_file_path, sep='\t', names = ["Label", "Message"])
print(test_data)

     Label                                            Message
0      ham  i am in hospital da. . i will return home in e...
1      ham         not much, just some textin'. how bout you?
2      ham  i probably won't eat at all today. i think i'm...
3      ham  don‘t give a flying monkeys wot they think and...
4      ham                                who are you seeing?
...    ...                                                ...
1387   ham  true dear..i sat to pray evening and felt so.s...
1388   ham               what will we do in the shower, baby?
1389   ham  where are you ? what are you doing ? are yuou ...
1390  spam  ur cash-balance is currently 500 pounds - to m...
1391  spam  not heard from u4 a while. call 4 rude chat pr...

[1392 rows x 2 columns]


In [40]:
train_data["Label"].value_counts()

ham     3619
spam     560
Name: Label, dtype: int64

In [41]:
test_data["Label"].value_counts()

ham     1205
spam     187
Name: Label, dtype: int64

In [42]:
train_data['Labeling'] = train_data['Label'].map({'ham': 1, 'spam':0})
train_data.tail()

,Label,Message,Labeling
4174,ham,just woke up. yeesh its late. but i didn't fal...,1
4175,ham,what do u reckon as need 2 arrange transport i...,1
4176,spam,free entry into our £250 weekly competition ju...,0
4177,spam,-pls stop bootydelious (32/f) is inviting you ...,0
4178,ham,tell my bad character which u dnt lik in me. ...,1


In [43]:
test_data['Labeling'] = test_data['Label'].map({'ham': 1, 'spam':0})
test_data.tail()

,Label,Message,Labeling
1387,ham,true dear..i sat to pray evening and felt so.s...,1
1388,ham,"what will we do in the shower, baby?",1
1389,ham,where are you ? what are you doing ? are yuou ...,1
1390,spam,ur cash-balance is currently 500 pounds - to m...,0
1391,spam,not heard from u4 a while. call 4 rude chat pr...,0


In [44]:
train_data['length'] = train_data['Message'].apply(len)
train_data.tail()

,Label,Message,Labeling,length
4174,ham,just woke up. yeesh its late. but i didn't fal...,1,74
4175,ham,what do u reckon as need 2 arrange transport i...,1,69
4176,spam,free entry into our £250 weekly competition ju...,0,106
4177,spam,-pls stop bootydelious (32/f) is inviting you ...,0,152
4178,ham,tell my bad character which u dnt lik in me. ...,1,165


In [45]:
test_data['length'] = test_data['Message'].apply(len)
test_data.tail()

,Label,Message,Labeling,length
1387,ham,true dear..i sat to pray evening and felt so.s...,1,75
1388,ham,"what will we do in the shower, baby?",1,36
1389,ham,where are you ? what are you doing ? are yuou ...,1,139
1390,spam,ur cash-balance is currently 500 pounds - to m...,0,148
1391,spam,not heard from u4 a while. call 4 rude chat pr...,0,160


In [46]:
X_train = train_data['Message']
X_test = test_data['Message']
Y_train = train_data['Labeling']
Y_test = test_data['Labeling']

# CountVectorizer
CountVectorizer is a great tool provided by the scikit-learn library in Python. It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

# Exclude Stop Words
vector = CountVectorizer(stop_words ='english')
vector.fit(X_train)

CountVectorizer(stop_words='english')

In [48]:
X_train_transformed = vector.transform(X_train)
X_test_transformed = vector.transform(X_test)

## Naive Bayes classifier for multinomial models

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.

In [49]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_transformed, Y_train)

y_pred = model.predict(X_test_transformed)

In [50]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test, y_pred))

0.9885057471264368


# Preprocess data

In [51]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
def text_processing(message):
    # Check characters to see if they are in punctuation
    non_punctuation = [char for char in message if char not in string.punctuation]

    # Join the characters again to form the string
    non_punctuation = ''.join(non_punctuation)
    
    # Now remove any stopwords
    return [word.lower() for word in non_punctuation.split() if word.lower() not in stopwords.words('english')]

"""
Takes in a string of text, then performs the following:
1. Remove all punctuation
2. Remove all stopwords
3. Return a list of the cleaned text with lowercase characters 
"""

'\nTakes in a string of text, then performs the following:\n1. Remove all punctuation\n2. Remove all stopwords\n3. Return a list of the cleaned text with lowercase characters \n'

In [53]:
training = train_data["Message"].apply(text_processing)
training

0       [ahhhhjust, woken, uphad, bad, dream, u, thoso...
1                                        [never, nothing]
2       [u, sound, like, manky, scouse, boy, stevelike...
3       [mum, say, wan, go, go, shun, bian, watch, da,...
4       [never, lei, v, lazy, got, wat, dat, day, ü, s...
                              ...                        
4174    [woke, yeesh, late, didnt, fall, asleep, til, ...
4175    [u, reckon, need, 2, arrange, transport, u, ca...
4176    [free, entry, £250, weekly, competition, text,...
4177    [pls, stop, bootydelious, 32f, inviting, frien...
4178    [tell, bad, character, u, dnt, lik, ill, try, ...
Name: Message, Length: 4179, dtype: object

In [54]:
testing = test_data["Message"].apply(text_processing)
testing

0                   [hospital, da, return, home, evening]
1                                    [much, textin, bout]
2       [probably, wont, eat, today, think, im, gonna,...
3       [don‘t, give, flying, monkeys, wot, think, cer...
4                                                [seeing]
                              ...                        
1387    [true, deari, sat, pray, evening, felt, soso, ...
1388                                       [shower, baby]
1389    [yuou, working, getting, pc, moms, find, spot,...
1390    [ur, cashbalance, currently, 500, pounds, maxi...
1391    [heard, u4, call, 4, rude, chat, private, line...
Name: Message, Length: 1392, dtype: object

In [55]:
training_emails = []
for email in training:
  list_to_str = ' '.join([str(word) for word in email])
  training_emails.append(list_to_str)

In [56]:
training_emails

['ahhhhjust woken uphad bad dream u thoso dont like u right didnt know anything comedy night guess im',
 'never nothing',
 'u sound like manky scouse boy stevelike travelling da bus homewot u inmind 4 recreation dis eve',
 'mum say wan go go shun bian watch da glass exhibition',
 'never lei v lazy got wat dat day ü send da url cant work one',
 'xam hall boy asked girl tell starting term dis answer den manage lot hesitation n lookin around silently said intha ponnungale ipaditan',
 'genius whats brother pls send number skype',
 'finally came fix ceiling',
 'urgent call 09066350750 landline complimentary 4 ibiza holiday 10000 cash await collection sae tcs po box 434 sk3 8wp 150 ppm 18',
 'started dont stop pray good ideas anything see help guys ill forward link',
 'hello darlin ive finished college txt u finish u love kate xxx',
 'aiyo cos sms ü ü neva reply wait 4 ü reply lar tot ü havent finish ur lab wat',
 'i\x92m cool ta luv vtired 2 cause doin loads planning wk got social services 

In [57]:
testing_emails = []
for email in testing:
  list_to_str = ' '.join([str(word) for word in email])
  testing_emails.append(list_to_str)

In [58]:
testing_emails

['hospital da return home evening',
 'much textin bout',
 'probably wont eat today think im gonna pop weekend u miss',
 'don‘t give flying monkeys wot think certainly don‘t mind friend mine',
 'seeing',
 'opinion 1 2 jada 3 kusruthi 4 lovable 5 silent 6 spl character 7 matured 8 stylish 9 simple pls reply',
 'yesterday going home',
 'yes innocent fun',
 'boy late 2 home father power frndship',
 'ur changes 2 da report big cos ive already made changes 2 da previous report',
 'sorry earlier putting firesare around talk 9 actually life lol',
 'im still looking car buy gone 4the driving test yet',
 'pls send address sir',
 'u got persons story',
 'love cuddle want hold strong arms right',
 'deam seeing online shop asked',
 'hey tell blakes address carlos wanted meet got lost hes answering phone',
 'yo im right yo work',
 'sister cleared two round birla soft yesterday',
 'fps',
 'finish already yar keep saying mushy embarrassed ok',
 'doinghow',
 'exwife able kids want kids one day',
 'yet 

# TfidfVectorizer
TfidfVectorizer transforms text to feature vectors that can be used as input to estimator. vocabulary is a dictionary that converts each token (word) to feature index in the matrix, each unique token gets a feature index.

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [60]:
vectorizer = TfidfVectorizer()
vectorizer.fit(training_emails)

X_train_transformed = vectorizer.transform(training_emails)
X_test_transformed = vectorizer.transform(testing_emails)

## Naive Bayes classifier for multinomial models

In [61]:
from sklearn.naive_bayes import MultinomialNB

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB()
spam_detect_model.fit(X_train_transformed, Y_train)

y_pred = spam_detect_model.predict(X_test_transformed)

In [62]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test, y_pred))

0.9626436781609196


# Testing

In [66]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = [pred_text]
  example = vector.transform(pred_text)
  pred = model.predict(example)
  if pred == 1:
    prediction = [1, 'ham'] 
  else:
    prediction = [0, 'spam'] 
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[1, 'ham']


In [67]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False
    print(prediction)

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


[1, 'ham']
[0, 'spam']
[1, 'ham']
[0, 'spam']
[0, 'spam']
[1, 'ham']
[1, 'ham']
You passed the challenge. Great job!
